<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 4*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Apply regularization techniques to your model. 

*Don't forgot to switch to GPU on Colab!*

## Regularization

Using your best performing model from the previous module, apply each of the following regularization strategies: 
* Early Stopping
* Dropout
* Weight Decay
* Weight Constraint


In [1]:
from tensorflow.keras.constraints import MaxNorm
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
from tensorflow.keras.layers import Flatten, Dense, Dropout
from tensorflow.keras.layers import ReLU
from tensorflow.keras.optimizers import SGD, Adam, Adadelta, Ftrl, Nadam
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp
from tensorflow.keras import Sequential
import tensorflow as tf
import seaborn as sns
import pandas as pd
import numpy as np
import datetime
import os

%reload_ext tensorboard

In [2]:
data = np.load('../quickdraw10.npz')
data.files
    
def load_quickdraw10(path, x_loc, y_loc, outlier_val=-1):
    """
    Get loaded data and split; returns X_train/test and y train_test
    with 40% split.
    Normalize data; enter in outlier value or keep default (-1) to 
    select auto and find X max.
    
    Input: data path, X and y index label
    
    :Return: X_train, X_test, y_train, y_test
    """
    X = path[x_loc]
    y = path[y_loc]
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y, 
                                                        test_size = 0.4,
                                                        random_state=445642)
    # Read or find outlier, -1 is auto, else inputed value used.
    if outlier_val == -1:
        outlier_val = X.max()
    else:
        outlier_val = outlier_val
    
    # Normalize X_test/train by outlier value
    X_train, X_test = X_train.astype('float32') / outlier_val, X_test.astype('float32') / outlier_val
    
    # Print info
    print(f'X shape = {X.shape}\n'
          f'y shape = {y.shape}\n'
          f'X (max, min) = ({X.max()}, {X.min()})\n'
          f'X (train.shape, test.shape) = [ {X_train.shape}, {X_test.shape} ]\n\n'
          f'y Unique = {len(np.unique(y))}')
    
    return X_train, y_train, X_test, y_test

# Set test and train sets
X_train, y_train, X_test, y_test = load_quickdraw10(path=data,
                                                   x_loc='arr_0',
                                                   y_loc='arr_1')

X shape = (100000, 784)
y shape = (100000,)
X (max, min) = (255, 0)
X (train.shape, test.shape) = [ (60000, 784), (40000, 784) ]

y Unique = 10


In [3]:
# 1) Create 2 dir for logging files 
# create 2 dir -- one for tensorboard results and one for early stopping
logdir = os.path.join("logs", 
                      "EarlyStopping+WeightConstraint+Dropout")

# 2) Instantiate the callbacks 
# instantiate a tensorboard callback object
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir)

# instantiate a early stopping clallback object 
# docs: https://keras.io/api/callbacks/early_stopping/
stop = EarlyStopping(monitor="val_loss", # track the progress of the test loss 
                     min_delta=0.001, # define threshold for what constitutes an improvment 
                     patience=5) # if an improvement doesn't happen after 3 epoches, stop training 


In [4]:
def create_model(X_train, X_test, y_train, 
                 y_test, hparams, opt=None, 
                 epochs=10, p_dropout=0.2):
    """
    SGD, Adam, Adadelta, Ftrl, nadam or hparams
    """
    if opt == None:
        opt = hparams[HP_OPTIMIZER]
    elif opt.lower() == 'sgd':
        opt = SGD(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'adam':
        opt = Adam(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'adadelta':
        opt = Adadelta(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'ftrl':
        opt = Ftrl(learning_rate=hparams[HP_LEARNING_RATE])
    elif opt.lower() == 'nadam':
        opt = Nadam(learning_rate=hparams[HP_LEARNING_RATE])
    else:
        raise ValueError(f"Unknown optimizer input of {opt} used")
    
    # Instanciate weight constraint
    wc = MaxNorm(max_value=2)
    
    # Model Creation
    model = tf.keras.Sequential([
        Flatten(),
        # Layer 1
        Dense(hparams[HP_NUM_UNITS], kernel_constraint=wc),
        ReLU(negative_slope=0.01),
        Dropout(p_dropout),
        # Layer 2
        Dense(64, kernel_constraint=wc),
        ReLU(negative_slope=0.1),
        Dropout(p_dropout),
        # Layer 3 
        Dense(32, kernel_constraint=wc),
        ReLU(negative_slope=0.01),
        Dropout(p_dropout),
        # Output Layer
        Dense(10, activation='softmax')
    ])
    
    model.compile(optimizer=opt,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    
    model.fit(X_train, y_train, 
              epochs=epochs,
              validation_data=(X_test, y_test),
              callbacks=[tensorboard_callback, stop])
        

    _, accuracy = model.evaluate(X_test, y_test)
    
    return accuracy

In [5]:
HP_NUM_UNITS = hp.HParam('num_units', hp.Discrete([16,32,100,200,300]))
HP_LEARNING_RATE = hp.HParam('learning_rate', hp.RealInterval(0.001,0.1))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))
METRIC_ACCURACY = 'accuracy'
GRID_SEARCH_RESULTS_DIR = 'logs/hparam_tuning'
# creating a dir to save/log our gridsearch results for use with tensorboard 
with tf.summary.create_file_writer(GRID_SEARCH_RESULTS_DIR).as_default():
    hp.hparams_config(
        # store h-params and their values 
        hparams=[HP_NUM_UNITS, HP_LEARNING_RATE, HP_OPTIMIZER],
        # store metrics to score the model 
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')])

In [6]:
def run(run_dir, hparams):

    with tf.summary.create_file_writer(run_dir).as_default():
        # record the values used in this trial
        hp.hparams(hparams)  

        # call create_model to build, train, and score model on parameter values 
        accuracy = create_model(
                                hparams=hparams, 
                                X_train=X_train, X_test=X_test,
                                y_train=y_train, y_test=y_test,
                                opt=None, epochs=20
                               )

        # store trained accuracy to file 
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)
        
session_num = 0 

for num_units in HP_NUM_UNITS.domain.values:
    for learning_rate in (HP_LEARNING_RATE.domain.min_value, HP_LEARNING_RATE.domain.max_value):
        for optimizer in HP_OPTIMIZER.domain.values:

            # as we loop through all the hyper-param values
            # the store each unique combination in the dictionary hparams
            hparams = {
               HP_NUM_UNITS: num_units,
               HP_LEARNING_RATE: learning_rate,
               HP_OPTIMIZER: optimizer
            }

            run_name = f"run-{session_num}"
            print(f"--- Starting trial: {run_name}")
            print({h.name: hparams[h] for h in hparams})

            # exectue the run function, which runs the training of the models 
            run('logs/hparam_tuning/' + run_name, hparams)
            session_num += 1

--- Starting trial: run-0
{'num_units': 16, 'learning_rate': 0.001, 'optimizer': 'adam'}
Epoch 1/20
1875/1875 [==============================] - 3s 1ms/step - loss: 1.4560 - accuracy: 0.4939 - val_loss: 0.8317 - val_accuracy: 0.7460
Epoch 2/20
1875/1875 [==============================] - 2s 987us/step - loss: 0.9845 - accuracy: 0.6820 - val_loss: 0.7609 - val_accuracy: 0.7663
Epoch 3/20
1875/1875 [==============================] - 2s 982us/step - loss: 0.9289 - accuracy: 0.7023 - val_loss: 0.7565 - val_accuracy: 0.7712
Epoch 4/20
1875/1875 [==============================] - 2s 997us/step - loss: 0.8974 - accuracy: 0.7154 - val_loss: 0.7264 - val_accuracy: 0.7821
Epoch 5/20
1875/1875 [==============================] - 2s 973us/step - loss: 0.8816 - accuracy: 0.7192 - val_loss: 0.7128 - val_accuracy: 0.7837
Epoch 6/20
1875/1875 [==============================] - 2s 970us/step - loss: 0.8660 - accuracy: 0.7270 - val_loss: 0.6925 - val_accuracy: 0.7911
Epoch 7/20
1875/1875 [===============

Epoch 14/20
1875/1875 [==============================] - 2s 961us/step - loss: 0.8368 - accuracy: 0.7408 - val_loss: 0.6754 - val_accuracy: 0.7951
Epoch 15/20
1875/1875 [==============================] - 2s 961us/step - loss: 0.8295 - accuracy: 0.7438 - val_loss: 0.6635 - val_accuracy: 0.8008
Epoch 16/20
1875/1875 [==============================] - 2s 951us/step - loss: 0.8224 - accuracy: 0.7445 - val_loss: 0.6674 - val_accuracy: 0.7998
Epoch 17/20
1875/1875 [==============================] - 2s 953us/step - loss: 0.8152 - accuracy: 0.7495 - val_loss: 0.6847 - val_accuracy: 0.7939
Epoch 18/20
1875/1875 [==============================] - 2s 949us/step - loss: 0.8209 - accuracy: 0.7452 - val_loss: 0.6747 - val_accuracy: 0.7946
Epoch 19/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.8268 - accuracy: 0.7471 - val_loss: 0.6732 - val_accuracy: 0.7976
Epoch 20/20
1250/1250 [==============================] - 1s 432us/step - loss: 0.6708 - accuracy: 0.7956
--- Starting tr

1875/1875 [==============================] - 2s 866us/step - loss: 0.8799 - accuracy: 0.7298 - val_loss: 0.7138 - val_accuracy: 0.7824
Epoch 7/20
1875/1875 [==============================] - 2s 876us/step - loss: 0.8520 - accuracy: 0.7377 - val_loss: 0.6935 - val_accuracy: 0.7886
Epoch 8/20
1875/1875 [==============================] - 2s 866us/step - loss: 0.8346 - accuracy: 0.7442 - val_loss: 0.6741 - val_accuracy: 0.7936
Epoch 9/20
1875/1875 [==============================] - 2s 863us/step - loss: 0.8055 - accuracy: 0.7565 - val_loss: 0.6661 - val_accuracy: 0.7977
Epoch 10/20
1875/1875 [==============================] - 2s 874us/step - loss: 0.8056 - accuracy: 0.7577 - val_loss: 0.6495 - val_accuracy: 0.8033
Epoch 11/20
1875/1875 [==============================] - 2s 871us/step - loss: 0.7818 - accuracy: 0.7665 - val_loss: 0.6429 - val_accuracy: 0.8050
Epoch 12/20
1875/1875 [==============================] - 2s 871us/step - loss: 0.7690 - accuracy: 0.7662 - val_loss: 0.6306 - val_acc

Epoch 20/20
1250/1250 [==============================] - 1s 439us/step - loss: 0.6022 - accuracy: 0.8187
--- Starting trial: run-8
{'num_units': 100, 'learning_rate': 0.001, 'optimizer': 'adam'}
Epoch 1/20
1875/1875 [==============================] - 3s 1ms/step - loss: 1.1616 - accuracy: 0.6215 - val_loss: 0.6062 - val_accuracy: 0.8149
Epoch 2/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.6806 - accuracy: 0.7983 - val_loss: 0.5551 - val_accuracy: 0.8333
Epoch 3/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.6096 - accuracy: 0.8177 - val_loss: 0.5161 - val_accuracy: 0.8433
Epoch 4/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5736 - accuracy: 0.8330 - val_loss: 0.5095 - val_accuracy: 0.8481
Epoch 5/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5476 - accuracy: 0.8367 - val_loss: 0.5098 - val_accuracy: 0.8453
Epoch 6/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5352 - 

Epoch 13/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4902 - accuracy: 0.8564 - val_loss: 0.4911 - val_accuracy: 0.8511
Epoch 14/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4858 - accuracy: 0.8560 - val_loss: 0.4742 - val_accuracy: 0.8546
Epoch 15/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4796 - accuracy: 0.8575 - val_loss: 0.4703 - val_accuracy: 0.8572
Epoch 16/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4765 - accuracy: 0.8571 - val_loss: 0.4754 - val_accuracy: 0.8568
Epoch 17/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4841 - accuracy: 0.8553 - val_loss: 0.4692 - val_accuracy: 0.8587
Epoch 18/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4679 - accuracy: 0.8607 - val_loss: 0.4659 - val_accuracy: 0.8586
Epoch 19/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4723 - accuracy: 0.8564 - val_loss: 0.4806 - val_ac

Epoch 11/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5682 - accuracy: 0.8328 - val_loss: 0.5085 - val_accuracy: 0.8457
Epoch 12/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5554 - accuracy: 0.8328 - val_loss: 0.4999 - val_accuracy: 0.8477
Epoch 13/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5367 - accuracy: 0.8405 - val_loss: 0.4906 - val_accuracy: 0.8511
Epoch 14/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5176 - accuracy: 0.8472 - val_loss: 0.4864 - val_accuracy: 0.8518
Epoch 15/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.5153 - accuracy: 0.8472 - val_loss: 0.4826 - val_accuracy: 0.8530
Epoch 16/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4982 - accuracy: 0.8512 - val_loss: 0.4769 - val_accuracy: 0.8548
Epoch 17/20
1875/1875 [==============================] - 2s 1ms/step - loss: 0.4824 - accuracy: 0.8566 - val_loss: 0.4709 - val_ac

Epoch 10/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.4013 - accuracy: 0.8778 - val_loss: 0.4407 - val_accuracy: 0.8680
Epoch 11/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3908 - accuracy: 0.8831 - val_loss: 0.4585 - val_accuracy: 0.8676
Epoch 12/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3845 - accuracy: 0.8854 - val_loss: 0.4609 - val_accuracy: 0.8669
Epoch 13/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3812 - accuracy: 0.8863 - val_loss: 0.4453 - val_accuracy: 0.8655
Epoch 14/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3753 - accuracy: 0.8884 - val_loss: 0.4351 - val_accuracy: 0.8704
Epoch 15/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3756 - accuracy: 0.8888 - val_loss: 0.4383 - val_accuracy: 0.8703
Epoch 16/20
1875/1875 [==============================] - 3s 2ms/step - loss: 0.3757 - accuracy: 0.8875 - val_loss: 0.4498 - val_ac

Epoch 12/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5205 - accuracy: 0.8443 - val_loss: 0.4854 - val_accuracy: 0.8516
Epoch 13/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.5118 - accuracy: 0.8488 - val_loss: 0.4769 - val_accuracy: 0.8543
Epoch 14/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4911 - accuracy: 0.8548 - val_loss: 0.4688 - val_accuracy: 0.8577
Epoch 15/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4798 - accuracy: 0.8582 - val_loss: 0.4641 - val_accuracy: 0.8595
Epoch 16/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4614 - accuracy: 0.8627 - val_loss: 0.4561 - val_accuracy: 0.8609
Epoch 17/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4428 - accuracy: 0.8691 - val_loss: 0.4522 - val_accuracy: 0.8646
Epoch 18/20
1875/1875 [==============================] - 3s 1ms/step - loss: 0.4396 - accuracy: 0.8708 - val_loss: 0.4531 - val_ac

## Deploy

Save your model's weights using the Checkpoint function. Try reloading the model and making inference on your validation dataset.

In [7]:
%tensorboard --logdir=logs/ --host localhost --port 8090

Reusing TensorBoard on port 8090 (pid 29788), started 0:53:12 ago. (Use '!kill 29788' to kill it.)

### Stretch Goals
- Mount your Google Drive to Colab to persist your model checkpoint files. 
- Research L2 normalization (weight decay)
- Write a custom callback function to stop training after you reach .88 validation accuracy. 
- Select a new dataset and apply a neural network to it.
- Research TensorFlow Serving
- Play [QuickDraw](https://quickdraw.withgoogle.com/data)
- Create a static webpage using TensorFlow.js to serve a model. Check out [Teachable Machine Learning](https://teachablemachine.withgoogle.com/) for ideas. 